In [1]:
import os
import glob
import numpy as np
import torch
import time
import cv2
#import json
import numpy as np
from PIL import Image
from torchvision.transforms import *
import torch.utils.data
import torch
import torch.nn as nn
from torch.autograd import Variable
from model import ConvColumn
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn


In [ ]:
### Loading model
# set run output folder
gpus = [0,0]
print("=> active GPUs: {}".format(gpus))

model_name = "jester_conv_example"

# create model
model = ConvColumn(27)

# multi GPU setting
model = torch.nn.DataParallel(model, device_ids= gpus).cuda()

if os.path.isfile("/home/hoanganh/Desktop/GulpIO-benchmarks-master/checkpoint.pth.tar"):
    print("=> loading checkpoint")
    checkpoint = torch.load("/home/hoanganh/Desktop/GulpIO-benchmarks-master/checkpoint.pth.tar")
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    print("=> loaded checkpoint (epoch {})"
          .format(checkpoint['epoch']))
else:
    print("=> no checkpoint found ")

cudnn.benchmark = False


In [ ]:
# declare initial parameters 

IMG_EXTENSIONS = ['.jpg', '.JPG', '.jpeg', '.JPEG']

clip_size = 18
nclips = 1
step_size = 2
is_val=True

# declare transform (crop, mean, std) #

transform = Compose([
    CenterCrop(84),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225])
])


###################################################

# root = './test_video/9223/%05d.jpg'%i     % path of frames

# define function to load image from a path of image file 
def image_loader(image_path):
    """load image, returns cuda tensor"""
    image = Image.open(image_path).convert('RGB')
    return image

# define function to take file name of frame in a forder

def get_frame_names(path):
    frame_names = []
    for ext in IMG_EXTENSIONS:
        frame_names.extend(glob.glob(os.path.join(path, "*" + ext)))
    frame_names = list(sorted(frame_names))
    num_frames = len(frame_names)
    # set number of necessary frames
    if nclips > -1:
        num_frames_necessary = clip_size * nclips * step_size
    else:
        num_frames_necessary = num_frames

    # pick frames
    offset = 0
    if num_frames_necessary > num_frames:
        # Pad last frame if video is shorter than necessary
        frame_names += [frame_names[-1]] * \
            (num_frames_necessary - num_frames)
    elif num_frames_necessary < num_frames:
        # If there are more frames, then sample starting offset.
        diff = (num_frames - num_frames_necessary)
        # temporal augmentation
        if not is_val:
            offset = np.random.randint(0, diff)
    frame_names = frame_names[offset:num_frames_necessary +
                              offset:step_size]
    return frame_names

In [ ]:
### get image paths from an input forder
start = time.time()

img_paths = get_frame_names('./test_dataset/109621/')

### get frames form img_paths 

imgs = []

for img_path in img_paths:
    img = image_loader(img_path)
    img = transform(img)
    imgs.append(torch.unsqueeze(img, 0))

# print(imgs.shape)    
## format data to torch
    
data = torch.cat(imgs)
data = data.permute(1, 0, 2, 3)
data = data.unsqueeze(0)
print(data.shape)
print(time.time() - start)

gestures = ['Swiping_Left', 'Swiping_Right', 'Swiping_Down', 'Swiping_Up',
            'Pushing_Hand_Away', 'Pulling_Hand_In', 'Sliding_Two_Fingers_Left',
            'Sliding_Two_Fingers_Right', 'Sliding_Two_Fingers_Down', 'Sliding_Two_Fingers_Up',
            'Pushing_Two_Fingers_Away', 'Pulling_Two_Fingers_In', 'Rolling_Hand_Forward', 
            'Rolling_Hand_Backward', 'Turning_Hand_Clockwise', 'Turning_Hand_Counterclockwise', 
            'Zooming_In_With_Full_Hand', 'Zooming_Out_With_Full_Hand', 
            'Zooming_In With_Two_Fingers', 'Zooming_Out_With_Two_Fingers', 
            'Thumb_Up', 'Thumb_Down', 'Shaking_Hand', 'Stop_Sign',
            'Drumming_Fingers', 'No_gesture', 'Doing_other_things']


In [ ]:
start = time.time()
input = data
# input_vars = torch.autograd.Variable(input.cuda(), volatile#=True)


input_vars = Variable(input)

output = model(input_vars)
_, predicted = torch.max(output.data, 1)

print(time.time() - start)
predicted = predicted.cpu().numpy()
predicted = predicted[0]


print('predict_gesture: {}'.format(i, gestures[predicted]))
print(time.time() - start)

input = input.cpu().numpy()


m1 = np.array([0.485, 0.456, 0.406])
s1 = np.array([0.229, 0.224, 0.225])


for t in range(18):
    cv2.imshow('input',np.array(255*(s1*np.transpose(np.array(input[0,:,t,:,:]),(1,2,0))+m1),dtype=np.uint8))
    cv2.waitKey(0)
cv2.destroyAllWindows()
